In [1]:
# We start with tudaraset
using Flux
using Flux: onecold, onehotbatch
using Flux.Losses: logitbinarycrossentropy
using Flux: DataLoader
using GraphNeuralNetworks
using MLDatasets: TUDataset
using Statistics, Random
using MLUtils
using CUDA
CUDA.allowscalar(false)

function eval_loss_accuracy(model, data_loader, device)
    loss = 0.0
    acc = 0.0
    ntot = 0
    for (g, y) in data_loader
        g, y = (g, y) |> device
        n = length(y)
        ŷ = model(g, g.ndata.x) |> vec
        loss += logitbinarycrossentropy(ŷ, y) * n
        acc += mean((ŷ .> 0) .== y) * n
        ntot += n
    end
    return (loss = round(loss / ntot, digits = 4),
            acc = round(acc * 100 / ntot, digits = 2))
end

eval_loss_accuracy (generic function with 1 method)

In [2]:
function getdataset()
    tudata = TUDataset("MUTAG")
    display(tudata)
    graphs = mldataset2gnngraph(tudata)
    oh(x) = Float32.(onehotbatch(x, 0:6))
    graphs = [GNNGraph(g, ndata = oh(g.ndata.targets)) for g in graphs]
    y = (1 .+ Float32.(tudata.graph_data.targets)) ./ 2
    @assert all(∈([0, 1]), y) # binary classification 
    return graphs, y
end

getdataset (generic function with 1 method)

In [3]:
# arguments for the `train` function 
Base.@kwdef mutable struct Args
    η = 1.0f-3             # learning rate
    batchsize = 32      # batch size (number of graphs in each batch)
    epochs = 200         # number of epochs
    seed = 17             # set seed > 0 for reproducibility
    usecuda = true      # if true use cuda (if available)
    nhidden = 128        # dimension of hidden features
    infotime = 10      # report every `infotime` epochs
end

Args

In [9]:
function train(; kws...)
    args = Args(; kws...)
    args.seed > 0 && Random.seed!(args.seed)

    if args.usecuda && CUDA.functional()
        device = gpu
        args.seed > 0 && CUDA.seed!(args.seed)
        @info "Training on GPU"
    else
        device = cpu
        @info "Training on CPU"
    end

    # LOAD DATA
    NUM_TRAIN = 150

    dataset = getdataset()
    train_data, test_data = splitobs(dataset, at = NUM_TRAIN, shuffle = true)

    train_loader = DataLoader(train_data; args.batchsize, shuffle = true, collate = true)
    test_loader = DataLoader(test_data; args.batchsize, shuffle = false, collate = true)

    # DEFINE MODEL

    nin = size(dataset[1][1].ndata.x, 1)
    nhidden = args.nhidden

    model = GNNChain(GraphConv(nin => nhidden, relu),
                     GraphConv(nhidden => nhidden, relu),
                     GlobalPool(mean),
                     Dense(nhidden, 1)) |> device

    opt = Flux.setup(Adam(args.η), model)

    # LOGGING FUNCTION

    function report(epoch)
        train = eval_loss_accuracy(model, train_loader, device)
        test = eval_loss_accuracy(model, test_loader, device)
        println("Epoch: $epoch   Train: $(train)   Test: $(test)")
    end

    # TRAIN

    report(0)
    for epoch in 1:(args.epochs)
        for (g, y) in train_loader
            g, y = (g, y) |> device
            grads = Flux.gradient(model) do model
                ŷ = model(g, g.ndata.x) |> vec
                logitbinarycrossentropy(ŷ, y)
            end
            Flux.update!(opt, model, grads[1])
        end
        epoch % args.infotime == 0 && report(epoch)
    end
end

train()

dataset TUDataset:
  name        =>    MUTAG
  metadata    =>    Dict{String, Any} with 1 entry
  graphs      =>    188-element Vector{MLDatasets.Graph}
  graph_data  =>    (targets = "188-element Vector{Int64}",)
  num_nodes   =>    3371
  num_edges   =>    7442
  num_graphs  =>    188

┌ Info: Training on CPU
└ @ Main c:\Users\HP\Desktop\VicWorks03082024\eSGCe_tudataset.ipynb:11


Epoch: 0   Train: (loss = 0.7044, acc = 24.0)   Test: (loss = 0.7066, acc = 31.58)
Epoch: 10   Train: (loss = 0.4826, acc = 74.67)   Test: (loss = 0.5597, acc = 65.79)
Epoch: 20   Train: (loss = 0.4376, acc = 77.33)   Test: (loss = 0.5244, acc = 68.42)
Epoch: 30   Train: (loss = 0.4159, acc = 79.33)   Test: (loss = 0.4941, acc = 78.95)
Epoch: 40   Train: (loss = 0.3899, acc = 79.33)   Test: (loss = 0.488, acc = 68.42)
Epoch: 50   Train: (loss = 0.3673, acc = 81.33)   Test: (loss = 0.4594, acc = 78.95)
Epoch: 60   Train: (loss = 0.377, acc = 80.0)   Test: (loss = 0.515, acc = 65.79)
Epoch: 70   Train: (loss = 0.3234, acc = 84.0)   Test: (loss = 0.4369, acc = 78.95)
Epoch: 80   Train: (loss = 0.3084, acc = 84.67)   Test: (loss = 0.4251, acc = 81.58)
Epoch: 90   Train: (loss = 0.2805, acc = 87.33)   Test: (loss = 0.416, acc = 81.58)
Epoch: 100   Train: (loss = 0.2689, acc = 86.0)   Test: (loss = 0.4196, acc = 78.95)
Epoch: 110   Train: (loss = 0.2543, acc = 88.67)   Test: (loss = 0.4119, 

In [10]:
function plot_predicted_data(graph,features,targets, sensor)
    p = plot(xlabel="Time (h)", ylabel="Normalized speed")
    prediction = []
    grand_truth = []
    for i in 1:3:length(features)
        push!(grand_truth,targets[i][1,sensor,:])
        push!(prediction, model(graph, features[i])[1,sensor,:]) 
    end
    prediction = reduce(vcat,prediction)
    grand_truth = reduce(vcat, grand_truth)
    plot!(p, collect(1:length(features)), grand_truth, color = :blue, label = "Grand Truth", xticks =([i for i in 0:50:250], ["$(i)" for i in 0:4:24]))
    plot!(p, collect(1:length(features)), prediction, color = :red, label= "Prediction")
    return p
end

plot_predicted_data (generic function with 1 method)

In [11]:
#plot_predicted_data(graph,features[301:588],targets[301:588], 1)

UndefVarError: UndefVarError: `graph` not defined